## DSCI 100 Group Project 

By Tony Fu, Hao Jiang, and Aimee Garcia Castro

## Introduction: 

There are many elements that play a role in how well a student will perform on a test. The objective for this group project is predicting the knowledge level of an individual based on such elements. The knowledge levels include: `high`, `middle`, `low` and `very_low` We will be looking at the "User Knowledge" dataset obtained from the UCI Machine Learning Repository. The data in this dataset looks at `STG` (the degree of study time for goal object materials), `SCG` (the degree of repetition number of user for goal object materials), `STR` (the degree of study time of user for related objects with goal object), `LPR` (the exam performance of user for related object with goal object), and `PEG` (the exam performance of user for goal objects). As well, it relates `PEG` to `UNS` (the knowledge level of the user). This data will be used as predictors for `UNS`
 
Our predictive question that we will try to answer is: “Can we predict the `UNS` of an individual based on factors such as `STG`, `SCG`, and `STR`?”
 


## Methods: 

We begin by loading the libraries required to perform exploratory analysis. 

In [4]:
library(dplyr)
library(tidyverse)
library(repr)
library(rvest)
library(stringr)
library(readxl)
library(ggplot2)
library(tidymodels)
library(GGally)
source("tests.R")
source("cleanup.R")
options(repr.matrix.max.rows = 6)

Warning message:
“package ‘dplyr’ was built under R version 4.0.2”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ stringr 1.4.0
✔ tidyr   1.1.2     ✔ forcats 0.5.0
✔ readr   1.3.1     

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘rvest’ was built under R version 4.0.2”
Loading required package: xml2


Attaching package: ‘rvest’


The following object is mask

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


Since the data we are going to use in our analysis is available as an Excel spreadsheet, we are going to use the `read_excel` function to transfer the data from the spreadsheet into R. We will be using the `sheet` function to specify which sheet of the Excel file contains the data we are going to use. This will be `sheet = 2` as the second sheet is listed as **training data**, which will be crucial once we create our classifier. 

We notice that the last 3 columns are irrelevant to the data, so we will be removing those by selecting the wanted data with the `select` function.

Furthermore, although not required, we chose to lowercase all the `UNS` labels to keep the labeling consistent with the `mutate` function.

In [14]:
user_knowledge <- read_excel("data/Data_User_Modeling_Dataset_Hamdi Tolga KAHRAMAN.xls", sheet = 2) %>%
    select(STG, SCG, STR, LPR, PEG, UNS) %>%
        mutate(UNS = recode(UNS, High = 'high', Middle = 'middle', Low = 'low'))

user_knowledge

New names:
* `` -> ...7
* `` -> ...8



STG,SCG,STR,LPR,PEG,UNS
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
0.00,0.000,0.00,0.00,0.00,very_low
0.08,0.080,0.10,0.24,0.90,high
0.06,0.060,0.05,0.25,0.33,low
0.10,0.100,0.15,0.65,0.30,middle
0.08,0.080,0.08,0.98,0.24,low
0.09,0.150,0.40,0.10,0.66,middle
0.10,0.100,0.43,0.29,0.56,middle
0.15,0.020,0.34,0.40,0.01,very_low
0.20,0.140,0.35,0.72,0.25,low


*Table 1*: Tidied "User Knowledge" dataset obtained from UCI Machine Learning Repository

## Searching for NAs


We will now be looking for NA values in our data frame, and handle them appropriately as they represent a missing or incomplete value in our data. 